## Extract Transform and Load the Data set.

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 

reviews_df = pd.read_csv("Reviews.csv")
helpfulnessMask = reviews_df["HelpfulnessNumerator"] > 0
reviews_df = reviews_df[helpfulnessMask]

reviews_df.drop(columns=["ProductId", "ProfileName", "HelpfulnessDenominator", "Time", "Summary"], inplace=True)

print(reviews_df.shape)
reviews_df.head()

(264628, 5)


,Id,UserId,HelpfulnessNumerator,Score,Text
0,1,A3SGXH7AUHU8GW,1,5,I have bought several of the Vitality canned d...
2,3,ABXLMWJIXXAIN,1,4,This is a confection that has been around a fe...
3,4,A395BORC6FGVXV,3,2,If you are looking for the secret ingredient i...
8,9,A1MZYO9TZK0BBI,1,5,Right now I'm mostly just sprouting this so my...
10,11,A3HDKO7OW0QNK4,1,5,I don't know if it's the cactus or the tequila...


## Sentiment Analysis

In [2]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax






#reviews_df.to_csv("transformed_reviews.csv", header=True, index=True)

/Users/yogeshsonik/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
